# Natural Language Processing Homework 1


<!DOCTYPE html>
<html lang="fa">
<head>
    <meta charset="UTF-8">
    <title>توضیح متن و تسک انجام شده</title>
</head>
<body>
    <p dir="rtl">
        در این تسک، ابتدا داده‌های مربوط به اخبار را در چهار دسته‌بندی ورزشی، اقتصادی، سیاسی و اجتماعی جمع‌آوری میکنیم. منبع داده‌های من از سایت رویداد ۲۴ میباشد و صفحات مختلف آن را توسط کتابخانه scrapy در پایتون، کرال کردم.
        <br>
        در ادامه از هر خبر عنوان و خلاصه آن خبر را استخراج کردم. حدود ۳.۷ مگابایت داده‌اولیه در اختیار داریم که باید آن را پردازش کنیم.
        <br>
        تسکی که به صورت کلی میخواهم روی داده‌ها انجام دهم بدین شکل هست که ابتدا عبارات کلیدی متن را استخراج نمایم و سپس یک بردار از کلمات کلیدی تشکیل دهم، و در نهایت با استفاده از ضرب برداری روی داده‌های تست کتگوری حدودی آن داده‌ها را مشخص نمایم.
    </p>
</body>
</html>


## install and import requirements

In [1]:
!pip install hazm 
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import json
from hazm import *
import random 
import numpy as np
import nltk
import pandas as pd
import codecs
import tqdm
import os

In [3]:
with open('news.json', 'r') as file:
  import json
  data = json.load(file)
  data = [(item['title'] + "\n" +item['content']) for item in data]

## Pre-Processing

In [4]:
data = [item.replace('\xa0', ' ') for item in data]
data

['قیمت جدید گوشت قرمز اعلام شد\nقائم مقام وزیر جهاد کشاورزی با اینکه مشکلی در تامین گوشت نداریم، گفت: بنابر برنامه ریزی صورت گرفته واردات گوشت قرمز به طور منظم در حال انجام است.',
 'کنفرانس خبری با کله و پاچه!\nرئیس اتحادیه اغذیه و پروتئین گلپایگان برای اثبات حرف\u200cهای خود تخلف برخی از طباخان این شهرستان با کله و پاچه به نشست خبری برگزار کرد.',
 'قیمت روز گوشی موبایل ۱۲ آبان ۱۴۰۲\nقیمت روز گوشی موبایل در بازار ۱۲ آبان ۱۴۰۲ را در جداول زیر ببینید.',
 'قیمت روز خودرو ۱۲ آبان ۱۴۰۲\nقیمت روز خودرو در بازار ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.',
 'قیمت روز ارز\u200cهای دیجیتال ۱۲ آبان ۱۴۰۲\nقیمت روز ارز\u200cهای دیجیتال در بازار ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.',
 'قیمت دلار، طلا و سکه امروز ۱۲ آبان ۱۴۰۲\nقیمت دلار، طلا و سکه در بازار امروز ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.',
 'قیمت روز خودرو\u200cهای خارجی؛ ماشینی که هر روز ۱۵۰ میلیون گران می\u200cشود!\nقیمت روز و جدید انواع خودرو\u200cهای خارجی در تاریخ ۱۲ آبان را می\u200cتوانید در ادامه ببینید.',
 'تمام نگاه\u200c دولت به شرق است/

### Normalize

In [5]:
normalizer = Normalizer()
news_normalized = [normalizer.normalize(item)for item in data]


for x in random.sample(news_normalized, 10):
    print(''.join(x))


خبرنگارانی که توسط اسرائیل کشته شدند +جدول
جمعه کاروانی از خبرنگاران چندین رسانه در جنوب لبنان مورد هدف موشک‌های اسرائیلی قرار گرفتند. در اثر این حمله یکی از خبرنگارات رویترز جانش را از دست داده و شش خبرنگار دیگر به شدت زخمی شدند. تاریخ نشان داده اسرائیل ید طولایی در حمله به رسانه‌ها و خبرنگاران دارد.
نمایی آرامش‌بخش از دامداری در روستاهای مازندران
تصاویری زیبا و دل‌انگیز از دامداری در روستاهای مازندران پیش روی شماست که می‌بینید.
چرا همسان‌سازی حقوق بازنشستگان از برنامه هفتم حذف شد؟ 
آنچه در کندوکاش در میان صفحات لایحه برنامه هفتم توسعه دیده می‌شود، این است که همسان‌سازی حقوق حذف‌شده و هیچ جایگاهی ندارد.
حقوق کارگران دوباره افزایش می‌یابد؟ 
تسنیم نوشت: اینطور که وزیرکار اعلام کرده است قرار است در هفته پیش رو موضوع ترمیم حقوق کارگران اطلاع‌رسانی شود، حالا باید دید مطابق درخواست جامعه کارگری حقوق کارگران ترمیم می‌شود یا خیر؟
انواع سنگ‌های ساختمانی با بهترین قیمت
سنگ‌های ساختمانی مانند گرانیت، تراوتن و مرمریت به عنوان مواد ساختمانی پرکاربرد در صنعت ساخت و ساز و اغلب سنگ نمای ساختمان استفا

### Tokenize

word tokenize and then sentence tokenize.

In [60]:
news_sentences = [sent_tokenize(item) for item in news_normalized]
news_tokens = [[word_tokenize(sent) for sent in sents] for sents in news_sentences]


for y in [item[0] for item in random.sample(news_tokens, 20) if item]:
    print(' '.join(y))

['قائم\u200cمقام وزیر جهاد کشاورزی با اینکه مشکلی در تامین گوشت نداریم، گفت: بنابر برنامه\u200cریزی صورت گرفته واردات گوشت قرمز به طور منظم در حال انجام است.', 'رئیس اتحادیه اغذیه و پروتئین گلپایگان برای اثبات حرف\u200cهای خود تخلف برخی از طباخان این شهرستان با کله و پاچه به نشست خبری برگزار کرد.', 'قیمت روز گوشی موبایل در بازار ۱۲ آبان ۱۴۰۲ را در جداول زیر ببینید.', 'قیمت روز خودرو در بازار ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.', 'قیمت روز ارز\u200cهای دیجیتال در بازار ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.', 'قیمت دلار، طلا و سکه در بازار امروز ۱۲ آبان ۱۴۰۲ را در جدول زیر ببینید.', 'قیمت روز و جدید انواع خودرو\u200cهای خارجی در تاریخ ۱۲ آبان را می\u200cتوانید در ادامه ببینید.', 'ذبیح\u200cالله اعظمی ساردویی نماینده جیرفت می\u200cگوید: دولت وزیر ضعیف انتخاب می\u200cکند و در نهایت یا خودش او را حذف می\u200cکند یا مجلس باید حذف کند که با این کار صدمه زیادی به کشور وارد می\u200cشود. هر وزیر جدیدی که می\u200cآید، تیم جدید می\u200cچیند و در این کارها، زمان و انرژی زیادی صرف می\u200cشود. چند ماه

### Frequency Analysis
We get words frequency in news. First, we exclude stop words from word tokens and then we use nltk to get frequency analysis of word tokens.

In [59]:
from nltk import FreqDist
import itertools
import pandas as pd


stopwords = [normalizer.normalize(x.strip()) for x in codecs.open('stop_words.txt','r','utf-8').readlines()]
all_tokens = list(itertools.chain(*itertools.chain(*news_tokens)))
all_tokens_non_stop = [t for t in all_tokens if t not in stopwords and len(t) > 1]

dataframe_nonstop = {}

dataframe_nonstop['rouydad24'] = FreqDist(all_tokens_non_stop).most_common(50)

freq_analysis_nonstop = pd.DataFrame(dataframe_nonstop)    
freq_analysis_nonstop

رئیس صداوسیما می‌گوید : در همه سیاست هایمان ، چون رسانه ملی هستیم نباید اجازه بدهیم کسی از ما باج‌خواهی کند و نه خودمان به کسی باج بدهیم .
به نظر می‌رسد جذب جلال‌الدین ماشاریپوف کاپیتان تیم ملی ازبکستان از سوی باشگاه استقلال منتفی شده_است .
محمودوند ، عضو کمیسیون امنیت و سیاست خارجی مجلس ، گلایه کرده_است « برخی آقایان تصمیم گیر در دولت و مجمع تشخیص مصلحت نظام و هر جای دیگر می‌گویند فقط مجلس روی به شفاف‌سازی بیاورد » .
یحیی گل محمدی با درخواستی جنجالی و عجیب از امیر قلعه نویی ، نشست خبری خود در بازی با الدحیل را تحت تاثیر قرار داد .
در دوره جدید سرمربیگری امیر قلعه نویی در تیم ملی ، ۱۰ بازیکن برای اولین بار توانسته‌اند تجربه بازی در این تیم را به دست آورند .
مصطفی پورمحمدی دبیر کل جامعه روحانیت مبارز گفت : فهم سیاسی و اجتماعی جامعه به خوبی در موارد متعدد نشان داده که مردم گول شعار‌ها و ظاهرسازی‌های کاندیدا‌ها را نمی‌خورند .
سرمربی تیم ملی فوتبال ایران تصمیم خود برای جایگزینی بازیکن مصدوم قلب دفاع این تیم در رقابت‌های جام ملت‌های آسیا را گرفته_است .
سید مجید رضوی آرانی اظهار کرد : در ستا

### Lemmatizer

In [24]:

lemmatizer = Lemmatizer()
all_tokens_nonstop_lemstem =   [lemmatizer.lemmatize(t) for t in all_tokens_non_stop]

all_tokens_lemstem =   [lemmatizer.lemmatize(t) for t in all_tokens]
dataframe_nonstop_lemstem = {}

dataframe_nonstop_lemstem['rouydad24.ir'] = FreqDist(all_tokens_nonstop_lemstem).most_common(50)

freq_analysis_nonstop_lemstem = pd.DataFrame(dataframe_nonstop_lemstem)   
freq_analysis_nonstop_lemstem

,rouydad24.ir
0,"(ایران, 468)"
1,"(بازار, 431)"
2,"(روز, 393)"
3,"(قیمت, 377)"
4,"(کشور, 339)"
5,"(دید#بین, 307)"
6,"(اسرائیل, 250)"
7,"(تهران, 230)"
8,"(تیم, 226)"
9,"(۱۴۰۲, 214)"


### Extract Most Common Nouns, Verbs, Adverbs and Adjectives
we do this by tagging every token and then analyzing the frequency of each word.

In [31]:
tagger = POSTagger(model='./resources/postagger.model')

In [39]:
import copy

dataframe_nonstop_lemstem_advjj = {}

selected = {"صفت ها": [], "قید ها" : [], "فعل ها" : [], "اسم ها" : []}
for x,y in FreqDist(all_tokens_nonstop_lemstem).most_common(1000):
    pos = tagger.tag([x])[0][1]
    if pos in ['AJ']:
        selected['صفت ها'].append((x,y)) 
    elif pos in ['N']:
        selected['اسم ها'].append((x,y)) 
    elif pos in ['Ne']:
        selected['قید ها'].append((x,y)) 
    elif pos in ['V']:
        selected['فعل ها'].append((x,y)) 

for key, value in selected.items() :
  dataframe_nonstop_lemstem_advjj[key]= copy.deepcopy(value[0:8])

dataframe_nonstop_lemstem_advjj = pd.DataFrame(dataframe_nonstop_lemstem_advjj) 
dataframe_nonstop_lemstem_advjj   

('تیم\u200cهای', 28)
('وزش', 22)
('قلعه', 13)
('درگذشت', 12)
('ضد', 10)
{'NUM', 'ADP,EZ', 'ADJ,EZ', 'CCONJ', 'PRON', 'ADV', 'NOUN', 'ADJ', 'NOUN,EZ', 'ADP', 'VERB'}


,صفت ها,قید ها,فعل ها,اسم ها
0,"(ملی, 154)","(امروز, 162)","(گذاشت#گذار, 52)","(ایران, 468)"
1,"(منتشر, 89)","(گذشته, 119)","(برد, 15)","(بازار, 431)"
2,"(آزاد, 71)","(تازه, 23)","(معتقدند, 14)","(روز, 393)"
3,"(جهانی, 65)","(امسال, 23)","(کشته, 14)","(قیمت, 377)"
4,"(سیاسی, 54)","(ابتدا, 16)","(رسیده, 13)","(کشور, 339)"
5,"(مجاز, 49)","(آخر, 16)","(خواست, 13)","(دید#بین, 307)"
6,"(مختلف, 48)","(درست, 16)","(لرزاند, 13)","(اسرائیل, 250)"
7,"(برتر, 48)","(کنون, 15)","(آورد#آور, 12)","(تهران, 230)"


## Most Common Phrases In Rouydad24.

In [62]:
def vasf_ext(sentence):
    grammar = r"""
      NVASF: {<N|Ne><AJ>}
      GHEIDV: {<ADV><V>}
      EZAFI: {<N|Ne><N|Ne>+}
    """
    cp = nltk.RegexpParser(grammar)
    return (cp.parse(sentence))


mosufs = []
gheids = []
ezafi = []
for sentences in news_tokens:
  for sentence in sentences:
    tree=vasf_ext(tagger.tag([word for word in sentence]))
    for subtree in tree.subtrees():
        if subtree.label() == 'NVASF':
            mosufs.append(subtree.leaves())
        if subtree.label() == 'EZAFI':
            ezafi.append(subtree.leaves())  

for sentences in news_tokens:
  for sentence in sentences:
    tree=vasf_ext(tagger.tag([lemmatizer.lemmatize(word) for word in sentence]))
    for subtree in tree.subtrees():
        if subtree.label() == 'GHEIDV':
            gheids.append(subtree.leaves())


('تیم\u200cهای', 28)
('وزش', 22)
('قلعه', 13)
('درگذشت', 12)
('ضد', 10)
{'NUM', 'ADP,EZ', 'ADJ,EZ', 'CCONJ', 'PRON', 'ADV', 'NOUN', 'ADJ', 'NOUN,EZ', 'ADP', 'VERB'}


,صفت ها,قید ها,فعل ها,اسم ها
0,"(ملی, 154)","(امروز, 162)","(گذاشت#گذار, 52)","(ایران, 468)"
1,"(منتشر, 89)","(گذشته, 119)","(برد, 15)","(بازار, 431)"
2,"(آزاد, 71)","(تازه, 23)","(معتقدند, 14)","(روز, 393)"
3,"(جهانی, 65)","(امسال, 23)","(کشته, 14)","(قیمت, 377)"
4,"(سیاسی, 54)","(ابتدا, 16)","(رسیده, 13)","(کشور, 339)"
5,"(مجاز, 49)","(آخر, 16)","(خواست, 13)","(دید#بین, 307)"
6,"(مختلف, 48)","(درست, 16)","(لرزاند, 13)","(اسرائیل, 250)"
7,"(برتر, 48)","(کنون, 15)","(آورد#آور, 12)","(تهران, 230)"


In [41]:
results = dict()
results['ترکیب های وصفی'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in mosufs]).most_common(50)
results['قید ها و فعل های مرکب'] = nltk.FreqDist( [' '.join([x[0],y[0]]) for x,y in gheids]).most_common(50)
results['ترکیب های اضافی'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in ezafi]).most_common(50)
results['کل عبارات'] = nltk.FreqDist( [' '.join([z[0] for z in item]) for item in ezafi + gheids + mosufs]).most_common(50)
pd.DataFrame(results)

,ترکیب های وصفی,قید ها و فعل های مرکب,ترکیب های اضافی,کل عبارات
0,"(جداول زیر, 111)","(چه شد#شو, 4)","(قیمت دلار, 59)","(جداول زیر, 111)"
1,"(جدول زیر, 66)","(قبل بود#باش, 4)","(قیمت روز, 47)","(جدول زیر, 66)"
2,"(رژیم صهیونیستی, 61)","(باز کرد#کن, 3)","(قیمت روز گوشی موبایل, 45)","(رژیم صهیونیستی, 61)"
3,"(ارز‌های دیجیتال, 45)","(پیش رفت#رو, 3)","(قیمت روز خودرو, 44)","(قیمت دلار, 59)"
4,"(فضای مجازی, 42)","(بیشتر شد#شو, 3)","(نوار غزه, 27)","(قیمت روز, 47)"
5,"(فیلم زیر, 22)","(چه بود#باش, 3)","(بورس تهران, 19)","(قیمت روز گوشی موبایل, 45)"
6,"(جمهوری اسلامی, 22)","(صرفا می‌توان, 2)","(بازار سهام, 18)","(ارز‌های دیجیتال, 45)"
7,"(گل محمدی, 22)","(باز گشت#گرد, 2)","(باشگاه استقلال, 17)","(قیمت روز خودرو, 44)"
8,"(شورای اسلامی, 21)","(چگونه بود#باش, 2)","(مدیریت بحران مخاطرات وضع هوا, 15)","(فضای مجازی, 42)"
9,"(تیم ملی, 19)","(همچنان پابرجاست, 2)","(نماینده مجلس, 14)","(نوار غزه, 27)"


,ترکیب های وصفی,قید ها و فعل های مرکب,ترکیب های اضافی,کل عبارات
0,"(جداول زیر, 111)","(چه شد#شو, 4)","(قیمت دلار, 59)","(جداول زیر, 111)"
1,"(جدول زیر, 66)","(قبل بود#باش, 4)","(قیمت روز, 47)","(جدول زیر, 66)"
2,"(رژیم صهیونیستی, 61)","(باز کرد#کن, 3)","(قیمت روز گوشی موبایل, 45)","(رژیم صهیونیستی, 61)"
3,"(ارز‌های دیجیتال, 45)","(پیش رفت#رو, 3)","(قیمت روز خودرو, 44)","(قیمت دلار, 59)"
4,"(فضای مجازی, 42)","(بیشتر شد#شو, 3)","(نوار غزه, 27)","(قیمت روز, 47)"
5,"(فیلم زیر, 22)","(چه بود#باش, 3)","(بورس تهران, 19)","(قیمت روز گوشی موبایل, 45)"
6,"(جمهوری اسلامی, 22)","(صرفا می‌توان, 2)","(بازار سهام, 18)","(ارز‌های دیجیتال, 45)"
7,"(گل محمدی, 22)","(باز گشت#گرد, 2)","(باشگاه استقلال, 17)","(قیمت روز خودرو, 44)"
8,"(شورای اسلامی, 21)","(چگونه بود#باش, 2)","(مدیریت بحران مخاطرات وضع هوا, 15)","(فضای مجازی, 42)"
9,"(تیم ملی, 19)","(همچنان پابرجاست, 2)","(نماینده مجلس, 14)","(نوار غزه, 27)"
